In [10]:
from mxnet import np, npx
from mxnet.gluon import nn

npx.set_np()



class Residual(nn.Block):  #@save
    """The Residual block of ResNet."""
    def __init__(self, num_channels, use_1x1conv=False, strides=1, **kwargs):
        super().__init__(**kwargs)
        self.conv1 = nn.Conv2D(num_channels, kernel_size=3, padding=1,
                               strides=strides)
        self.conv2 = nn.Conv2D(num_channels, kernel_size=3, padding=1)
        if use_1x1conv:
            self.conv3 = nn.Conv2D(num_channels, kernel_size=1,
                                   strides=strides)
        else:
            self.conv3 = None
        self.bn1 = nn.BatchNorm()
        self.bn2 = nn.BatchNorm()

    def forward(self, X):
        Y=self.conv1(X)
        Y=self.bn1(Y)
        Y = self.relu(Y)
        Y = self.bn2(self.conv2(Y))
        if self.conv3:
            X = self.conv3(X)
        return self.relu(Y + X)
    
    def relu(self,X):
        return nd.maximum(X, 0)

In [32]:
import mxnet as mx 

class MyDense(nn.Block):
    ####################
    # We add arguments to our constructor (__init__)
    # to indicate the number of input units (``in_units``)
    # and output units (``units``)
    ####################
    def __init__(self, units, in_units=0, **kwargs):
        super(MyDense, self).__init__(**kwargs)
        self.units = units
        self._in_units = in_units
        self.weight = self.params.get(
                'weight', init=mx.init.Xavier(magnitude=2.24),shape=(in_units, units))
        self.bias = self.params.get('bias', shape=(units,))

    #################
    #  Now we just have to write the forward pass.
    #  We could rely upong the FullyConnected primitive in NDArray,
    #  but it's better to get our hands dirty and write it out
    #  so you'll know how to compose arbitrary functions
    #################
    def forward(self, x):
        with x.context:
            linear = nd.dot(x, self.weight.data()) + self.bias.data()
            activation = self.relu(linear)
            return activation
        
    def relu(self,X):
        return nd.maximum(X, 0)

In [33]:
dense = MyDense(20, in_units=10)
dense.initialize(ctx=mx.cpu())


TypeError: get() takes no keyword arguments

In [36]:
dense.params

{}

In [31]:
from mxnet import nd, autograd, gluon
from mxnet.gluon import nn, Block

dense(nd.ones(shape=(2,10)).as_in_context(mx.cpu()))

AttributeError: 'MyDense' object has no attribute 'weight'

In [5]:
net = nn.Sequential()
net.add()

In [6]:
def resnet_block(num_channels, num_residuals, first_block=False):
    blk = nn.Sequential()
    for i in range(num_residuals):
        if i == 0 and not first_block:
            blk.add(Residual(num_channels, use_1x1conv=True, strides=2))
        else:
            blk.add(Residual(num_channels))
    return blk

In [7]:
net.add(resnet_block(64, 2, first_block=True), resnet_block(128, 2),
        resnet_block(256, 2), resnet_block(512, 2))

In [8]:
net.add(nn.GlobalAvgPool2D(), nn.Dense(10))

In [11]:
net.collect_params()

{'0.weight': Parameter (shape=(64, 1, 7, 7), dtype=<class 'numpy.float32'>),
 '0.bias': Parameter (shape=(64,), dtype=<class 'numpy.float32'>),
 '1.gamma': Parameter (shape=(-1,), dtype=<class 'numpy.float32'>),
 '1.beta': Parameter (shape=(-1,), dtype=<class 'numpy.float32'>),
 '1.running_mean': Parameter (shape=(-1,), dtype=<class 'numpy.float32'>),
 '1.running_var': Parameter (shape=(-1,), dtype=<class 'numpy.float32'>),
 '4.0.conv1.weight': Parameter (shape=(64, -1, 3, 3), dtype=<class 'numpy.float32'>),
 '4.0.conv1.bias': Parameter (shape=(64,), dtype=<class 'numpy.float32'>),
 '4.0.conv2.weight': Parameter (shape=(64, -1, 3, 3), dtype=<class 'numpy.float32'>),
 '4.0.conv2.bias': Parameter (shape=(64,), dtype=<class 'numpy.float32'>),
 '4.0.bn1.gamma': Parameter (shape=(-1,), dtype=<class 'numpy.float32'>),
 '4.0.bn1.beta': Parameter (shape=(-1,), dtype=<class 'numpy.float32'>),
 '4.0.bn1.running_mean': Parameter (shape=(-1,), dtype=<class 'numpy.float32'>),
 '4.0.bn1.running_var'

In [10]:
X = np.random.uniform(size=(1, 1, 224, 224))
net.initialize()
for layer in net:
    X = layer(X)
    #print(layer.name, 'output shape:\t', X.shape)

/home/sergio/code/mxnet/python/mxnet/gluon/block.py:575: UserWarning: Parameter 'weight' is already initialized, ignoring. Set force_reinit=True to re-initialize.
  v.initialize(None, ctx, init, force_reinit=force_reinit)
/home/sergio/code/mxnet/python/mxnet/gluon/block.py:575: UserWarning: Parameter 'bias' is already initialized, ignoring. Set force_reinit=True to re-initialize.
  v.initialize(None, ctx, init, force_reinit=force_reinit)


TypeError: convolution() got an unexpected keyword argument 'name'